In [1]:
!pip install gensim

In [2]:
# モジュール読み込み
import csv
import pickle

import gensim
from gensim import models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
dataset_file_name = './dataset/corpus.csv'
model_file_name = 'logistic.pkl'

In [4]:
# csvファイルを扱いやすいフォーマットに変形

with open(dataset_file_name, 'r') as f:
    data = list(csv.reader(f))

texts, label_ids = [], []
label2id = {}
idx_label, idx_sentence = 0, 1
sum_words = []

for counter, row in enumerate(data):
    if counter == 0:
        continue
    label = row[idx_label]
    if label not in label2id:
        label2id[label] = len(label2id)
    label_ids.append(label2id[label])
    word_list = row[idx_sentence].split(' ')
    texts.append(word_list)
    
    for line in word_list:
        for word in line:
            sum_words.append(word)

print('sum_words: ', len(sum_words))
id2label = {v: k for k, v in label2id.items()}

sum_words:  102532


In [5]:
#　テキストとラベルのデータをtrain, testに分割します
X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, label_ids, test_size=0.2, random_state=42)

In [6]:
# trainのテキストデータから、tfidfで重み付けされた単語文書行列を作成します

# テキストデータから辞書を作成します
dictionary = gensim.corpora.Dictionary(X_train_texts)
# 辞書を用いてBoW形式に文章を行列化します
corpus = [dictionary.doc2bow(text) for text in X_train_texts]

# BoW形式で作成したcorpusをtfidfを用いて重み付けします
tfidf_model = models.TfidfModel(corpus)
tfidf_corpus = tfidf_model[corpus]

num_words = len(dictionary)
X_train_tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words).T

In [7]:
# testのテキストデータから、tfidfで重み付けされた単語文書行列を作成します

# 辞書を用いてBoW形式に文章を行列化します
corpus = [dictionary.doc2bow(text) for text in X_test_texts]
# BoW形式で作成したcorpusをtfidfを用いて重み付けします
tfidf_corpus = tfidf_model[corpus]

num_words = len(dictionary)
X_test_tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words).T

In [8]:
# trainデータを用いて分類器を構築します
clf = LogisticRegression(solver='lbfgs')

In [9]:
clf.fit(X_train_tfidf, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# testデータを用いて分類器の精度を評価します
y_pred = clf.predict(X_test_tfidf)
target_names = list(id2label.values())

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

       dazai       0.72      0.95      0.82       732
        mori       0.94      0.66      0.78       492
   akutagawa       0.87      0.72      0.79       551

    accuracy                           0.80      1775
   macro avg       0.84      0.78      0.79      1775
weighted avg       0.83      0.80      0.80      1775

[[698   8  26]
 [132 327  33]
 [143  13 395]]
